# Note by the author (Muneeb)
# You would need to install the spectral library for this. Also give the 
# hyperspectral path and make sure that the hdr directory also
# has the .raw file with the same name as the .hdr file.

In [ ]:
import spectral
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# change teh following based on your computer
hdr_path = r"C:\Users\mm17889\OneDrive - University of Georgia\Hyperspectral Data\emptyname_2024-04-17_19-42-01\capture\emptyname_2024-04-17_19-42-01.hdr"

img = spectral.open_image(hdr_path)  # The .hdr points to the .raw automatically.

# Load into a NumPy array (shape typically [rows, cols, bands]):
data = img.load()  # or img.asarray()

print("Data shape (rows, cols, bands):", data.shape)


In [ ]:
if 'wavelength' in img.metadata:
    wavelengths = np.array([float(w) for w in img.metadata['wavelength']])
    print("Wavelength array shape:", wavelengths.shape)
else:
    wavelengths = None
    print("No wavelength info found in header.")

def find_band_index(target_nm):
    if wavelengths is None:
        raise ValueError("No wavelength data to match band indices.")
    return np.argmin(np.abs(wavelengths - target_nm))


In [ ]:
# Approximate wavelength values (nm) for each color band
blue_wl  = 450
green_wl = 550
red_wl   = 660
nir_wl   = 850

# Find actual band indices based on nearest wavelength:
if wavelengths is not None:
    blue_idx  = find_band_index(blue_wl)
    green_idx = find_band_index(green_wl)
    red_idx   = find_band_index(red_wl)
    nir_idx   = find_band_index(nir_wl)
else:
    # Fallback if no wavelength info is available
    blue_idx  = 10
    green_idx = 30
    red_idx   = 50
    nir_idx   = 90

print("Band indices (B, G, R, NIR):", blue_idx, green_idx, red_idx, nir_idx)


In [ ]:
# Typically we order as [R, G, B] for normal color display
rgb_bands = [red_idx, green_idx, blue_idx]
rgb_img = data[:, :, rgb_bands]

# Normalize for display (clip outliers at the 99th percentile)
rgb_max = np.percentile(rgb_img, 99)
rgb_norm = np.clip(rgb_img / rgb_max, 0, 1)

plt.figure(figsize=(6, 6))
plt.imshow(rgb_norm)
plt.title("RGB Composite")
plt.axis("off")
plt.show()


In [ ]:
# NDVI = (NIR - RED) / (NIR + RED + epsilon)
red_band = data[:, :, red_idx].astype(np.float32)
nir_band = data[:, :, nir_idx].astype(np.float32)

epsilon = 1e-8  # small offset to avoid division by zero
ndvi = (nir_band - red_band) / (nir_band + red_band + epsilon)

plt.figure(figsize=(6, 6))
plt.imshow(ndvi, cmap='RdYlGn')
plt.title("NDVI")
plt.colorbar(label="NDVI value")
plt.axis("off")
plt.show()


In [ ]:
# visualizing individual bands
# Red band (grayscale)
plt.figure(figsize=(6, 6))
plt.imshow(red_band, cmap='gray')
plt.title("Red Band")
plt.colorbar(label="Pixel Intensity")
plt.axis("off")
plt.show()

# NIR band (grayscale)
plt.figure(figsize=(6, 6))
plt.imshow(nir_band, cmap='gray')
plt.title("NIR Band")
plt.colorbar(label="Pixel Intensity")
plt.axis("off")
plt.show()
